# Исследование надежности заемщиков


**Описание проекта**

Заказчик исследования - кредитный отдел банка.<br>
Цель исследования - определить влияние семейного положения и количества детей клиента на своевременное погашение кредита. На основе полученных результатов разработать рекомендации по включению семейного положения и количества детей в модель кредитного скоринга.
<br>
Входные данные от банка — статистика о платёжеспособности клиентов.<br>

## Общая информация о данных

Импортируем библиотеку pandas. Считаем данные из csv-файла в датафрейм и сохраним в переменную `data`.

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

Выведем первые 20 строчек датафрейма `data` на экран

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Выведем основную информацию о датафрейме с помощью метода `info()`

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

Выведем количество пропущенных значений для каждого столбца. Используем комбинацию двух методов.

In [4]:
data.isna().sum()

,0
children,0
days_employed,2174
dob_years,0
education,0
education_id,0
family_status,0
family_status_id,0
gender,0
income_type,0
debt,0


В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце обработаем на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()`.

In [6]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

,days_employed
income_type,
безработный,366413.652744
в декрете,3296.759962
госслужащий,2689.368353
компаньон,1547.382223
пенсионер,365213.306266
предприниматель,520.848083
сотрудник,1574.202821
студент,578.751554


У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть.

Выведем перечень уникальных значений столбца `children`.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски заполнены. Проверим себя и ещё раз выведем количество пропущенных значений для каждого столбца с помощью двух методов.

In [12]:
data.isna().sum()

,0
children,0
days_employed,0
dob_years,0
education,0
education_id,0
family_status,0
family_status_id,0
gender,0
income_type,0
debt,0


### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру.

In [14]:
data['education'] = data['education'].str.lower()

Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


Используем собственную функцию с именем `categorize_income()` и метод `apply()`.

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

<ipython-input-18-0bc713f3049f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['total_income_category'] = data['total_income'].apply(categorize_income)


Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

Используем собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучим данные в столбце `purpose` и определим, какие подстроки помогут правильно определить категорию.**

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Исследование данных

#### ****Гипотезы****

Для исследования влияния семейного положения и количества детей клиента на факт погашения кредита в срок можно выдвинуть несколько гипотез.

***Гипотеза 1***: Клиенты, состоящие в браке, чаще погашают кредит в срок по сравнению с холостыми, разведёнными или вдовцами.
	Семейное положение может отражать уровень стабильности и ответственности клиента. Брак может указывать на наличие совместного бюджета и большей финансовой ответственности.

***Гипотеза 2***: Клиенты с большим количеством детей имеют более низкий уровень погашения кредита в срок по сравнению с клиентами без детей или с одним ребёнком.
	Клиенты с несколькими детьми могут испытывать больше финансовых трудностей, чем бездетные клиенты или с одним ребенком, что может снижать вероятность погашения кредита в срок.
    
***Гипотеза 3***: Клиенты с более высоким уровнем дохода имеют более высокую вероятность возврата кредита в срок по сравнению с клиентами с более низким уровнем дохода по причине того, что имеют больше финансовых ресурсов для своевременного погашения кредита.

    

    




#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [22]:
# создадим сводною таблицу с индексом "children", подсчитаем общее количество клиентов и количество должников
data_children = data.pivot_table(index = ['children'], values = 'debt', aggfunc = {'count','sum'})
# создадим колонки с расчетом доли
data_children['share'] = round(data_children['sum']/data_children['count'] * 100, 2)
# сортируем по убыванию значений столбца "Доля"
data_children = data_children.sort_values(by='share', ascending=False)
# переименование стоблцов сводной таблицы
data_children = data_children.rename(columns={'count':'Всего клиентов',
                                        'sum':'Кол-во должников', 'share':'Доля должников, %'})

display(data_children)

,Всего клиентов,Кол-во должников,"Доля должников, %"
children,,,
4,41,4,9.76
2,2052,194,9.45
1,4808,444,9.23
3,330,27,8.18
0,14091,1063,7.54
5,9,0,0.00


**Вывод:** Анализ групп клиентов по количеству детей показывает, что семьи с 1-4 детьми имеют наибольшую долю должников, что может быть связано с увеличением расходной части их бюджета. Напротив, семьи без детей демонстрируют наименьшую долю должников (7,5%), что указывает на их способность своевременно обслуживать кредиты без дополнительных финансовых нагрузок. Группа клиентов с 5 детьми не имеет должников, однако малый размер выборки (всего 9 клиентов) делает этот вывод менее надежным. Важно отметить, что при анализе данных необходимо учитывать несбалансированность выборок, так как малые выборки могут привести к искажению результатов и необоснованным выводам.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
# создадим сводною таблицу с индексом "family_status",
# подсчитаем общее количество клиентов и количество должников в каждой группе
data_status = data.pivot_table(index = ['family_status'], values = 'debt', aggfunc = {'count','sum'})
# создадим колонки с расчетом доли
data_status['share'] = round(data_status['sum']/data_status['count'] * 100, 2)
#  сортируем по убыванию значений столбца "Доля"
data_status = data_status.sort_values(by='share', ascending=False)
# переименуем столбцы
data_status = data_status.rename(columns={'count':'Всего клиентов',
                                        'sum':'Кол-во должников', 'share':'Доля должников, %'})
display(data_status)

,Всего клиентов,Кол-во должников,"Доля должников, %"
family_status,,,
Не женат / не замужем,2796,273,9.76
гражданский брак,4134,385,9.31
женат / замужем,12261,927,7.56
в разводе,1189,84,7.06
вдовец / вдова,951,63,6.62


**Вывод:** Группа "Не женат/не замужем" имеет долю должников выше, чем остальные группы, что может указывать на более высокий риск просрочки кредита в этой категории.
Женатые и замужние клиенты имеют более низкую долю должников, что может свидетельствовать о большей финансовой ответственности и возможности погасить кредит в срок.

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [25]:
# создадим сводною таблицу с индексом 'total_income_category' (доход),
# подсчитаем общее количество клиентов и количество должников в каждой группе
data_income = data.pivot_table(index = ['total_income_category'], values = 'debt', aggfunc = {'count','sum'})
# создадим колонки с расчетом доли
data_income['share'] = round(data_income['sum']/data_income['count'] * 100, 2)
#  сортируем по убыванию значений столбца "Доля"
data_income = data_income.sort_values(by='share', ascending=False)
# переименуем столбцы
data_income = data_income.rename(columns={'count':'Всего клиентов',
                                        'sum':'Кол-во должников', 'share':'Доля в %'})
display(data_income)

,Всего клиентов,Кол-во должников,Доля в %
total_income_category,,,
E,22,2,9.09
C,15921,1353,8.50
A,25,2,8.00
B,5014,354,7.06
D,349,21,6.02


**Вывод:** Представленные выборки несбалансированы и какой-то вывод можно делать только по двум категориям с достаточным размером выборок.

Самую высокую долю должников имеет группа с самыми низкими доходами (0–30000). Однако, следует заметить, что в этой группе меньше всего наблюдений, их следует перепроверить на большей выборке.
Самой ответсвенной является категория 'D' (30001–50000). Средние показатели по возврату кредита - у групп со средним доходом и выше.

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [ ]:
# создание сводной таблицы с индексом 'purpose_category',
# подсчет общего количества клиентов и количества должников в каждой группе
data_purpose = data.pivot_table(index = ['purpose_category'], values = 'debt', aggfunc = {'count','sum'})
# создание колонки и расчет доли
data_purpose['share'] = round(data_purpose['sum']/data_purpose['count'] * 100, 2)
#  сортировка по убыванию значений столбца "Доля"
data_purpose = data_purpose.sort_values(by='share', ascending=False)
# переименование столбцов
data_purpose = data_purpose.rename(columns={'count':'Всего клиентов',
                                        'sum':'Кол-во должников', 'share':'Доля должников, %'})
display(data_purpose)

**Вывод:** Чаще не возвращают кредит в срок клиенты, бравшие его для покупки автомобиля или для получения образования. Возможно, кредиты на автомобиль и обучение несут дополнительные риски связанные или с возможной аварией, или с проблемами трудоустройства после обучения.

Операции с недвижимостью имеют наименьшую долю должников, что может свидетельствовать о более стабильной финансовой ситуации клиентов, берущих кредиты на недвижимость.

#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

*Ответ:* Существует несколько причин отсутствия данных в датасетах, в том числе:

Человеческий фактор: Ошибки ввода данных, не желание отвечать на определенные вопросы.

Технические проблемы: Возможны сбои с оборудованием или программным обеспечением для сбора данных. Отсутствующие данные могут быть удалены в процессе обработки или очистки данных.

Организационные: Конфиденциальность, могут отсутствовать данные, идентифицирующие отдельных лиц или компании.

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* Пропуски принято заполнять медианным значением, поскольку оно более устойчиво к выбросам, чем среднее значение. Это позволяет сохранить исходное распределение данных и обеспечивает более надежные результаты при дальнейшей обработке и анализе.

### Общий вывод.

В ходе исследования надёжности заемщиков были данные от банка — статистика о платёжеспособности клиентов. Данные были изучены и обработаны:
- пропуски в данных о доходах заменены медианным значением по каждому типу занятости;
- заменены все отрицательные значения положительными с помощью метода abs();
- удалены анамальные значения в столбце children;
- заполнены пропуски в столбце days_employed медианными значениями по каждого типа занятости;
- заменен вещественный тип данных в столбце total_income на целочисленный с помощью метода astype();
- неявные дубликаты в столбце education приведены их к нижнему регистру;
- удалены строки-дубликаты в данных;
- проведена категоризации в столбцах данных о доходах и целях кредита;
- сформулированы гипотезы исследования.


На основе примененных шагов по обработке данных можно дать следующие рекомендации по улучшению сбора данных:
- сбор в одном формате (например, только положительные числа для доходов),
- валидация данных на этапе ввода, чтобы избежать отрицательных значений или других некорректных форматов;
- введение четкой категоризации данных для целей кредита;
- использовать для анализа большее количество наблюдений и репрезентативные выборки.

Исходя из представленных в наше распоряжение данных, с учетом их обработки (заполнения пропусков, удаления дублей и тд), можно сделать вывод что наиболее ответсвенными в отношении возврата кредита являются клиенты с доходом 30001–50000, состоящие в браке и не имеющие детей, берущие кредит на приобретение недвижимости. И напротив, склонны нарушать сроки погашения люди, не состояшие никогда в браке, имеющие 4 детей, и имеющие или низкие доходы, или, напротив, очень высокие, берущие кретит для покупки автомобиля или получения образования.

Таким образом, выдвинутые гипотезы частично подтверждены. Однако, нужно большее количество данных, более репрезентативные выборки, чтоб делать однозначные выводы.